In [347]:
using StaticArrays
using BenchmarkTools
using Interpolations
# using Plots
# plotly()

INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/AxisAlgorithms.ji for module AxisAlgorithms.
INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/Interpolations.ji for module Interpolations.


In [58]:
include("src/BinaryRegionTrees.jl")

brt

In [59]:
include("test/runtests.jl")

Test Summary:        | Pass  Total
  parents and children |   15     15
Test Summary: | 

Pass  Total
  geometry    |    8      8


trees_tests

```
1D:

c[2] * x + c[1] * (1 - x)

2D:

[c[1,2] * y + c[1,1] * (1 - y);
 c[2,2] * y + c[2,1] * (1 - y]
 
= c[:,2] * y + c[:,1] * (1 - y)
```

In [338]:
module interp

using StaticArrays

slicelast(A::SVector, i::Integer) = A[i]
slicelast(A::SMatrix, i::Integer) = A[:,i]

slicelast{T <: StaticArray}(A::T, i::Integer) = slicelast(A, size(T), i)

@generated function slicelast{Dims}(A::SArray{Dims}, size::NTuple{3}, i::Integer)
    newsize = Dims[1:(end-1)]
    stride = prod(newsize)
    quote
        @assert i >= 1 && i <= $(Dims[end])
        start = (i - 1) * $stride
        @inbounds result = SMatrix{$(newsize...)}(($([:(A[$j + start]) for j in 1:stride]...)))
        result
    end
end 

@generated function slicelast{Dims}(A::SArray{Dims}, size::NTuple, i::Integer)
    newsize = Dims[1:(end-1)]
    stride = prod(newsize)
    quote
        @assert i >= 1 && i <= $(Dims[end])
        start = (i - 1) * $stride
        @inbounds result = SArray{$(newsize)}(($([:(A[$j + start]) for j in 1:stride]...)))
        result
    end
end

# TODO: delete all other slicelast() implementations once slicing
# an SArray returns a fixed-size type
@generated function slicelast{T, N}(A::AbstractArray{T, N}, i::Integer)
    Expr(:ref, :A, [:(:) for j in 1:(N - 1)]..., :i)
end

function interpolate(corner_values::AbstractVector, normalized_point::AbstractVector)
    normalized_point[1] * (corner_values[2] - corner_values[1]) + corner_values[1]
end

function interpolate{T, N}(corner_values::AbstractArray{T, N}, normalized_point::AbstractVector)
    c1 = slicelast(corner_values, 1)
    c2 = slicelast(corner_values, 2)
    y = normalized_point[N]
    reduced_corners = (c2 - c1) * y + c1
    interpolate(reduced_corners, pop(normalized_point))
end

@generated function interp2{T, N}(values::AbstractArray{T, N}, normalized_point::SVector{N})
    exprs = Expr[]
    for (i, I) in enumerate(CartesianRange(ntuple(_ -> 2, Val{N})))
        terms = [Expr(:ref, :values, i)]
        for j in 1:N
            if I[j] == 1
                push!(terms, :((1 - normalized_point[$j])))
            else
                push!(terms, :((normalized_point[$j])))
            end
        end
        push!(exprs, Expr(:call, :(*), terms...))
    end
    quote
        @assert length(values) == $(2^N)
        @inbounds result = $(Expr(:call, :(+), exprs...))
        result
    end
end

end
    

interp

In [365]:
c = SVector(1.0, 2.0)
p = SVector(0.25)
println(@benchmark interp.interpolate($c, $p))
interp.interpolate(c, p)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.972 ns (0.00% GC)
  median time:      1.978 ns (0.00% GC)
  mean time:        2.023 ns (0.00% GC)
  maximum time:     13.575 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%


1.25

In [366]:
@benchmark interp.interp2($c, $p)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.974 ns (0.00% GC)
  median time:      7.172 ns (0.00% GC)
  mean time:        8.389 ns (0.00% GC)
  maximum time:     98.570 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [367]:
itp = interpolate(c, BSpline(Linear()), OnGrid())

2-element Interpolations.BSplineInterpolation{Float64,1,Array{Float64,1},Interpolations.BSpline{Interpolations.Linear},Interpolations.OnGrid,0}:
 1.0
 2.0

In [369]:
@benchmark $itp[$p[1] + 1]

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.426 ns (0.00% GC)
  median time:      6.751 ns (0.00% GC)
  mean time:        7.248 ns (0.00% GC)
  maximum time:     32.212 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [370]:
c = SMatrix{2,2}(1,2,3,4)

2×2 StaticArrays.SMatrix{2,2,Int64,4}:
 1  3
 2  4

In [371]:
p = SVector(0.25, 0.25)
println(@benchmark interp.interpolate($c, $p))
interp.interpolate(c, p)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     22.254 ns (0.00% GC)
  median time:      22.531 ns (0.00% GC)
  mean time:        24.091 ns (0.00% GC)
  maximum time:     114.222 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     996
  time tolerance:   5.00%
  memory tolerance: 1.00%


1.75

In [372]:
println(@benchmark interp.interp2($c, $p))
interp.interp2(c, p)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.855 ns (0.00% GC)
  median time:      8.356 ns (0.00% GC)
  mean time:        8.463 ns (0.00% GC)
  maximum time:     23.958 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999
  time tolerance:   5.00%
  memory tolerance: 1.00%


1.75

In [374]:
itp = interpolate(c, BSpline(Linear()), OnGrid())
itp[(p + 1)...]

1.75

In [375]:
@benchmark $itp[$p[1] + 1, $p[2] + 1]

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     13.466 ns (0.00% GC)
  median time:      14.748 ns (0.00% GC)
  mean time:        15.336 ns (0.00% GC)
  maximum time:     60.786 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     998
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [386]:
c = SArray{(2,2,2)}(1:8)
p = SVector(0.25, 0.5, 0.5)
println(@benchmark interp.interpolate($c, $p))
interp.interpolate(c, p)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     44.476 ns (0.00% GC)
  median time:      48.527 ns (0.00% GC)
  mean time:        54.181 ns (0.00% GC)
  maximum time:     290.535 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     990
  time tolerance:   5.00%
  memory tolerance: 1.00%


4.25

In [387]:
@benchmark interpolate!($c, BSpline(Linear()), OnGrid())

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.871 ns (0.00% GC)
  median time:      6.164 ns (0.00% GC)
  mean time:        6.860 ns (0.00% GC)
  maximum time:     38.895 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [389]:
itp = interpolate!(c, BSpline(Linear()), OnGrid())
itp[(p + 1)...]

4.25

In [390]:
@benchmark $itp[$p[1] + 1, $p[2] + 1, $p[3] + 1]

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     42.996 ns (0.00% GC)
  median time:      44.323 ns (0.00% GC)
  mean time:        45.153 ns (0.00% GC)
  maximum time:     154.435 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     990
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [391]:
itp.coefs

2×2×2 StaticArrays.SArray{(2,2,2),Int64,3,8}:
[:, :, 1] =
 1  3
 2  4

[:, :, 2] =
 5  7
 6  8

In [392]:
@benchmark size($itp)

BenchmarkTools.Trial: 
  memory estimate:  32.00 bytes
  allocs estimate:  1
  --------------
  minimum time:     80.932 ns (0.00% GC)
  median time:      89.466 ns (0.00% GC)
  mean time:        99.343 ns (2.58% GC)
  maximum time:     6.210 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     966
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [394]:
@which size(itp)

size{T,N}(itp::Interpolations.AbstractInterpolation{T,N,IT<:Union{Interpolations.InterpolationType,Tuple{Vararg{Interpolations.InterpolationType,N<:Any}}},GT<:Union{Interpolations.GridType,Interpolations.NoInterp,Tuple{Vararg{Union{Interpolations.GridType,Interpolations.NoInterp},N<:Any}}}}) at /Users/rdeits/.julia/v0.5/Interpolations/src/Interpolations.jl:66

In [393]:
@code_warntype size(itp)

Variables:
  #self#::Base.#size
  itp::Interpolations.BSplineInterpolation{Float64,3,StaticArrays.SArray{(2,2,2),Int64,3,8},Interpolations.BSpline{Interpolations.Linear},Interpolations.OnGrid,0}
  #1::Interpolations.##1#2{Interpolations.BSplineInterpolation{Float64,3,StaticArrays.SArray{(2,2,2),Int64,3,8},Interpolations.BSpline{Interpolations.Linear},Interpolations.OnGrid,0}}

Body:
  begin 
      #1::Interpolations.##1#2{Interpolations.BSplineInterpolation{Float64,3,StaticArrays.SArray{(2,2,2),Int64,3,8},Interpolations.BSpline{Interpolations.Linear},Interpolations.OnGrid,0}} = $(Expr(:new, Interpolations.##1#2{Interpolations.BSplineInterpolation{Float64,3,StaticArrays.SArray{(2,2,2),Int64,3,8},Interpolations.BSpline{Interpolations.Linear},Interpolations.OnGrid,0}}, :(itp)))
      SSAValue(0) = #1::Interpolations.##1#2{Interpolations.BSplineInterpolation{Float64,3,StaticArrays.SArray{(2,2,2),Int64,3,8},Interpolations.BSpline{Interpolations.Linear},Interpolations.OnGrid,0}}
      SSAVal

In [381]:
typeof(itp)

Interpolations.BSplineInterpolation{Float64,3,Array{Float64,3},Interpolations.BSpline{Interpolations.Linear},Interpolations.OnGrid,0}

In [345]:
interp.interp2(c, p)

4.25

In [346]:
@benchmark interp.interp2($c, $p)

BenchmarkTools.Trial: 
  memory estimate:  112.00 bytes
  allocs estimate:  7
  --------------
  minimum time:     48.078 ns (0.00% GC)
  median time:      56.110 ns (0.00% GC)
  mean time:        69.957 ns (11.09% GC)
  maximum time:     3.043 μs (93.19% GC)
  --------------
  samples:          10000
  evals/sample:     988
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [330]:
brt.ASDFs.interpolate(c, 0.25, 0.5, 0.5)

4.25

In [ ]:
interp.interpolate(c, SVector(0.25, 0.25))

In [ ]:
interp.interpolate(c, SVector(0.5, 0.5))

In [15]:
A = rand((2,3,4))

2×3×4 Array{Float64,3}:
[:, :, 1] =
 0.263339  0.333819  0.0193581
 0.532203  0.858432  0.856775 

[:, :, 2] =
 0.87074   0.481755   0.563134
 0.281816  0.0394651  0.410987

[:, :, 3] =
 0.426685  0.290138  0.0850805
 0.545889  0.437831  0.445268 

[:, :, 4] =
 0.807086  0.386907  0.695631
 0.459884  0.713543  0.516528

In [19]:
@code_warntype interp.slicelast(A, 2)

Variables:
  #self#::interp.#slicelast
  A::Array{Float64,3}
  i::Int64

Body:
  begin  # line 6:
      SSAValue(18) = interp.:
      SSAValue(19) = interp.:
      SSAValue(20) = i::Int64
      # meta: location abstractarray.jl getindex 752
      SSAValue(15) = SSAValue(18)
      SSAValue(16) = SSAValue(19)
      SSAValue(17) = SSAValue(20)
      $(Expr(:inbounds, false))
      # meta: location multidimensional.jl _getindex 270
      $(Expr(:boundscheck, true))
      SSAValue(2) = (Core.tuple)(SSAValue(15),SSAValue(16),SSAValue(17))::Tuple{Colon,Colon,Int64}
      $(Expr(:inbounds, false))
      # meta: location abstractarray.jl checkbounds 284
      # meta: location abstractarray.jl checkbounds 270
      # meta: location abstractarray.jl indices 67
      # meta: location array.jl size 20
      # meta: location array.jl _size 24
      SSAValue(9) = (Base.arraysize)(A::Array{Float64,3},(Base.box)(Int64,(Base.add_int)(0,1)))::Int64
      # meta: location array.jl _size 24
      SSAValue(

In [54]:
cell = adf[1,1,1][2,1,1][2,1,1][2,1,1]

Cell: Oct.HyperRectangle{3,Float64}([-2.5,-10.0,-10.0],[2.5,2.5,2.5])

In [41]:
interpolate(cell, Oct.center(cell.boundary))

13.302397285592981

In [42]:
s(Oct.center(cell.boundary))

13.272975363497064

In [44]:
p = SVector(0.01, 0.01, 0.01)
@benchmark Oct.findleaf($adf, $p)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     265.216 ns (0.00% GC)
  median time:      291.348 ns (0.00% GC)
  mean time:        301.750 ns (0.00% GC)
  maximum time:     986.351 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     305
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [ ]:
plt = scatter3d(1, xlim=(-10,10), ylim=(-10,10), zlim=(-10,10), marker=1)
queue = [adf]
i = 0
while !isempty(queue) && i < 1000
    i += 1
    cell = pop!(queue)
    for vert in Oct.vertices(cell.boundary)
        push!(plt, vert...)
    end
    if !Oct.isleaf(cell)
        append!(queue, Oct.children(cell))
    end
end
plt